### TODO: realizar generalização na copia do dataset e nao na referencia

In [ ]:
import random as rdm
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("salario.csv")
df.head()

In [ ]:
df.describe()

## Unindo os atributos de localidade

In [ ]:
ids = df.id
cities = df.cidade
states = df.estado
countries = df.pais

In [ ]:
locality = []
for i in range(len(cities)):
    locality.append([ids[i], str('{}, {}, {}'.format(cities[i], states[i], countries[i]))])

In [ ]:
df_locality = pd.DataFrame(locality, columns=['id', 'localidade'])
df_locality.head()

## Unindo os dataframes

In [ ]:
join_dataset = pd.concat([df, df_locality], axis=1, join='inner')
join_dataset.head()

## Removendo atributos

In [ ]:
dataset = join_dataset.drop(columns = ['id', 'pais', 'estado', 'cidade'])
dataset.describe()

## Função de generalização da data

In [ ]:
def date_generalization(dataset, level):
    if level == 1:
        # date '**/MM/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**' + dataset.iloc[data]['data'][2:10])
    elif level == 2:
        # date '**/**/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**' + dataset.iloc[data]['data'][5:10])
    elif level == 3:
        # date'**/**/****'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**/****')
        
    return dataset

In [ ]:
def locality_generalization(dataset, level):
    if level == 0:
        # locality 'city'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = locais[0]+","+locais[1]+","+locais[2]
    elif level == 1:
        # locality 'state,'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,"+locais[1]+","+locais[2]
    elif level == 2:
        # locality 'country'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,**,"+locais[2]
    elif level == 3:
        # locality '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = '**'
        
    return dataset

In [ ]:
def gender_generalization(dataset, level):
    if level == 1:
        # gender '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['genero'] = '*'
            
    return dataset

## Função de Precisão

$precision (D, Na, h, HGVA) = 1 - \frac{\sum_{i=1}^{Na} \sum_{j=1}^{|D|} \frac{h}{HGVA_i}}{|D| * Na}
$

- D: conjunto de dados
- Na: número de atributos semi-identificadores
- h: altura da hierarquia de generalização de valor do atributo Ai após anonimização
- |HGVAi|: altura máxima da hierarquia

In [ ]:
def precision(D, Na, h, HGVA):
    summation = 0
    for i in range(Na):
        for j in range(len(D)):
            summation += (h[i] / (HGVA[i]))
    
    return 1 - (summation / (len(D) * Na))

In [ ]:
def soma(x,y):
    print()

# Gerar dados K-anonimizados

In [ ]:
def anonymize(df_faltante, k):

    d = {'data': [], 'genero': [], 'localidade':[], 'salario':[]}
    df_final = pd.DataFrame(data=d)

    Ordem_Generalizacoes = [(date_generalization, 1),
                            (locality_generalization, 1),
                            (date_generalization, 2),
                            (locality_generalization, 2),
                            (gender_generalization, 1),
                            (date_generalization, 3),
                            (locality_generalization, 3),
                            (soma, 1)]
    generalizations = {0: 0, 1: 0, 2: 0}
    index_generalization = 0
    for generalizacao_Atula in Ordem_Generalizacoes:
        flag = False
        aux = df_faltante.groupby(["data","genero","localidade"])["localidade"].count()

        atributos_K_anonimizados = aux[aux >= k].index

        for i in atributos_K_anonimizados:
            flag = True
            data = i[0]
            genero = i[1]
            localidade = i[2]

            indices_prontos = df_faltante[ (df_faltante['data'] == data) & (df_faltante['genero'] == genero) & (df_faltante["localidade"] == localidade) ].index#[0]

            df_final = df_final.append( df_faltante.iloc[indices_prontos], ignore_index = True )
            df_faltante.drop(indices_prontos , inplace=True)
            df_faltante = df_faltante.reset_index(drop=True)
            
        if flag:  
            if index_generalization == 0 or index_generalization == 2 or index_generalization == 5:
                generalizations[0] += 1

            if index_generalization == 1 or index_generalization == 3 or index_generalization == 6:
                generalizations[2] += 1

            if index_generalization == 4:
                generalizations[1] += 1
            
        index_generalization += 1
        
        generalizacao_Atula[0](df_faltante, generalizacao_Atula[1])
    
    print('Precision for k =', k, 'is', precision(df_final, 3, generalizations, [4, 1, 3]))
    df_final.to_csv (r'output/export_dataframe_k_' + str(k) +'.csv', index = None, header=True)


In [ ]:
for k in [2, 4, 8, 16]:
    anonymize(dataset.copy(), k)